In [1]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
import pandas as pd
import time
import os
import json
from tqdm import tqdm
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
def mailContent():
    SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        df = pd.DataFrame()
        service = build("gmail", "v1", credentials=creds) # Call the Gmail API
        twelve_hours_ago = int(time.time()) - 12 * 3600  # Filter emails from last 12 hours
        query = f"after:{twelve_hours_ago}"
        results = (
            service.users().messages().list(userId="me", labelIds=["INBOX"], q=query).execute()
        )
        messages = results.get("messages", [])

        if not messages:
            print("No messages found.")
            return
        #print("Messages:")
        i=0
        for message in messages:
            df.loc[i,'MessageId'] = message["id"]
            msg = (
                service.users().messages().get(userId="me", id=message["id"]).execute()
            )
            payload = msg['payload']
            headers = payload['headers']
            #print(headers)
            df.loc[i,"Date"] = next((header['value'] for header in headers if header['name'] == 'Date'), None)
            df.loc[i,'Subject'] = next((header['value'] for header in headers if header['name'] == 'Subject'), None)
            df.loc[i,'sender'] = next((header['value'] for header in headers if header['name'] == 'From'), None)
            
            parts = payload.get('parts')
            if parts and 'data' in parts[0].get('body', {}):
                data = parts[0]['body']['data']
                df.loc[i,'body'] = base64.urlsafe_b64decode(data).decode('utf-8')
            else:
                body = payload['body'].get('data')
                if body:
                    df.loc[i,'body'] = base64.urlsafe_b64decode(body).decode('utf-8')
                else:
                    df.loc[i,'body'] = ''
            i=i+1
        return df
    except HttpError as error:
        print(f"An error occurred: {error}")
        return



In [3]:
result=mailContent()
##take in the body & subject -> Identify the Weather it is a Job related email -> If yes Update check if the csv already has a entry _> if not create a new entry 
##                                                                                                                                   -> If yes read and add the update
## The tracker CSV needs -> Applied Date, Company Name, Role, Status, Last Updated
result

,MessageId,Date,Subject,sender,body
0,1987f47b2824265a,"Wed, 06 Aug 2025 12:07:41 +0000",📚20% OFF gift cards📚,Panera Bread <panera@m1.panerabread.com>,All the meals to make their year!\n\n\n\n\n\n\...
1,1987ee9e1740cfab,"Wed, 6 Aug 2025 10:24:15 +0000",🤫 Why arguing facts makes people believe harder,Quiet Moves <quietmoves@substack.com>,View this post on the web at https://quietmove...


In [4]:
load_dotenv()
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION="2023-05-15"
#print(AZURE_OPENAI_API_KEY)
#print(AZURE_OPENAI_ENDPOINT)

In [5]:
def get_user_prompt(body,subject):
    
    #Function to Generate the Full User Prompt
    
    prompt = f'''
    Below is an email received by a user.

    Subject:
    "{subject}"

    Body:
    "{body}"

    Your task is to determine whether this email is related to a job application. Specifically, check if it includes:

    - A confirmation of a job application submission
    - A status update regarding the application
    - A rejection notice
    - An interview schedule notification

    If it is related, return ONLY the following **raw JSON** object—without markdown, text, or any extra formatting:

    {{
        "Company Name": "Name of the company or organization",
        "Role": "The job title or role the user applied for",
        "Status": "Current status of the job application"
    }}

    **Valid values for "Status" are:**
    - "Applied"
    - "Interview Scheduled"
    - "Rejected"

    If the email is not related to a job application, return NOTHING (not even an empty string, explanation, or JSON).
    '''
    return prompt

In [6]:
def model_response(sys_prompt,user_prompt):
    
    #Function to Generate  User Response using OPENAI
    client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    api_version="2024-08-01-preview",)
    response = client.chat.completions.create(
    model="gpt-35-turbo-16k",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ])
    content = response.choices[0].message.content.strip()
    # Basic validation before parsing
    if not content.startswith("{"):
        print("Model returned non-JSON output:", content)
        return None

    try:
        return json.loads(content)
    except json.JSONDecodeError as e:
        print(" JSON parsing error:", e)
        print("Returned content:", content)
        return None

In [7]:
def refresh_status():
    # Load emails
    emails_df = mailContent()

    # Initialize the tracking DataFrame
    df = pd.DataFrame({'AppliedDate': pd.Series(dtype='str'),'CompanyName': pd.Series(dtype='str'),'Role': pd.Series(dtype='str'),
                    'Status': pd.Series(dtype='str'),'LastUpdated': pd.Series(dtype='str')})

    # System prompt for the model
    sys_prompt = (
        "You are a helpful Job Application Tracker Assistant. "
        "Your role is to identify whether an email is related to a job application and, "
        "if so, extract a clear status update based on the email content in JSON format."
    )

    for index in range(len(emails_df)):
        print(f'{index + 1}/{len(emails_df)}')
        user_prompt = get_user_prompt(emails_df.loc[index, 'Subject'],emails_df.loc[index, 'body'])
        response = model_response(sys_prompt, user_prompt)
        print(response)
        if not response or not isinstance(response, dict):
            print(f"Skipping index {index} due to invalid response: {response}")
            continue

        # Match on CompanyName and Role (more reliable than just CompanyName)
        mask = (
            (df["CompanyName"] == response.get("Company Name")) &
            (df["Role"] == response.get("Role"))
        )

        if df[mask].empty:
            # Add a new row
            new_row = pd.DataFrame([{"AppliedDate": emails_df.loc[index, "Date"],"CompanyName": response.get("Company Name"),"Role": response.get("Role"),"Status": response.get("Status"),
                "LastUpdated": emails_df.loc[index, "Date"]}])
            df = pd.concat([df, new_row], ignore_index=True)
        else:
            # Update the existing row
            df.loc[mask, ['Status', 'LastUpdated']] = [
                response.get('Status'), emails_df.loc[index, 'Date']
            ]

    return df

In [8]:
result=refresh_status()
result.head()

1/2


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-08-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}

In [66]:
result

,MessageId,Date,Subject,sender,body
0,1985c221a1be5682,2025-07-30 16:19:54+00:00,Your Application to EcoDataLab,Gusto <gustonoreply@gusto.com>,<p>Thank you for your interest in the Climate ...
1,1985bb027d2fb128,2025-07-30 14:15:27+00:00,L'Oréal Invites You To Our Next Masterclass: H...,Tangela Woodley via Handshake <handshake@notif...,Tangela just messaged you about an event\r\n\r...
2,1985bada045e1692,2025-07-30 14:12:41+00:00,Today's Lesson: Back-to-School Savings ✏️,SHEIN <info@news.us.shein.com>,
3,1985b95d502b3061,2025-07-30 13:46:41+00:00,"Shruti, don’t miss these newly released courses",LinkedIn <messages-noreply@linkedin.com>,"Rutgers University\r\n \r\nShruti, don’t miss..."
4,1985b8276258fdb1,2025-07-30 13:25:28+00:00,10% Off >> Score July's Best Sellers,Rutgers Scarlet Knights Spirit Shop <shop@e.fa...,10% Off >> Score July's Best Sellers\r\nhttps...
5,1985b404906a74ae,2025-07-30 12:13:15+00:00,2 Months FREE sips ends soon!,MyPanera <panera@m1.panerabread.com>,Get the deal before it's gone...\n\n\n\n\n\n\n...
6,1985add8e97d9e90,2025-07-30 10:25:26+00:00,The latest jobs picked for you!,eFinancialCareers <emails@emails.efinancialcar...,"<!doctype html><html lang=""en"" xmlns=""http://w..."
